## 自己紹介{.smaller}
:::: {.columns}

::: {.column width="50%"}
### 略歴
- 福井県立藤島高校
- 京都大学医学部医学科
    - B3

### 東進歴
- 2020/10〜 物理科添削者
- 2020/11〜 物理チームリーダー
- 2021/04〜 ITチーム加入
:::

::: {.column width="50%"}
### プログラミング歴
- Python歴 1年半くらい
- Go歴 数ヶ月
- その他勉強中
    - TypeScript, Rust, Haskell
:::

::::

## 目次
- Pythonと型アノテーション
- typing関連豆知識n連発

## Pythonと型アノテーション
- 型アノテーションは実行時の挙動に影響を与えない
- Python <-> 型アノテーション付きPython
    - 型の静的解析は[mypy](https://github.com/python/mypy)や[pyright](https://github.com/microsoft/pyright)が行う
    - 両者は微妙に互換性がないので, 厄介...([参考](https://future-architect.github.io/articles/20220301a/))
- JavaScript <-> TypeScript
    - 型の静的解析はトランスパイラであるtscが行う

## typing関連豆知識n連発{.smaller}
### `NewType` (3.11<=)
- 部分型であることを型チェッカーに教える
- ドメイン固有型で属性を一つも持たないときとか...？

In [6]:
#| echo: true
from typing import NewType
# Money <: int
Money = NewType("Money", int)
def add_money(a: Money, b: Money) -> Money:
    return a + b

In [11]:
#| echo: true
add_money(Money(1), Money(2))

3

In [13]:
#| echo: true
# 実行時エラーにはならない
add_money(1, 2)

3

- mypyでエラーを検出
    - ` error: Argument 1 to "add_money" has incompatible type "int"; expected "Money"  [arg-type]`

## typing関連豆知識n連発{.smaller}
### `Protocol` (3.8<=)
- TypeScriptやGoのような構造的部分型(structual subtyping)を表現可能に
- 明示的に親クラスを継承しなくても, 属性が対応していれば部分型とみなす
- static duck typing

In [22]:
#| echo: true
from typing import Protocol
class IPerson(Protocol):
    def walk(self) -> None:
        pass
class ImplPerson:
    def walk(self) -> None:
        print(f"I will walk!")
class NoImplPerson:...

def call_walkable(person: IPerson):
    person.walk()

In [23]:
#| echo: true
call_walkable(ImplPerson())

I will walk!


## typing関連豆知識n連発{.smaller}
### `Protocol` (3.8<=)

In [ ]:
#| echo: true
from typing import Protocol
class IPerson(Protocol):
    def walk(self) -> None:
        pass
class ImplPerson:
    def walk(self) -> None:
        print(f"I will walk!")
class NoImplPerson:...

def call_walkable(person: IPerson):
    person.walk()

In [24]:
#| echo: true
call_walkable(NoImplPerson())

AttributeError: 'NoImplPerson' object has no attribute 'walk'

- Protocolを満たしていないクラスのインスタンスへの呼び出しはmypyでもエラーになる
    - `error: Argument 1 to "call_walkable" has incompatible type "NoImplPerson"; expected "IPerson"  [arg-type]`

## typing関連豆知識n連発{.smaller}
### `Literal` (3.8<=)
- TypeScriptでの`type fooOrBar: "foo" | "bar"`のようなことができる

In [35]:
#| echo: true
from typing import Literal
Subject = Literal["math", "english", "japanese"]
def get_score(subject: Subject) -> int:
    # Subject型以外の文字列だとバグるかもしれない関数
    if subject == "physics": raise ValueError("エラーだよ")

In [36]:
#| echo: true
get_score("math")

In [37]:
#| echo: true
get_score("physics")

ValueError: エラーだよ

- 動かす前に, mypyでエラーを見つけることができる
    - `error: Argument 1 to "get_score" has incompatible type "Literal['physics']"; expected "Literal['math', 'english', 'japanese']"  [arg-type]`

## typing関連豆知識n連発{.smaller}
### `TypedDict` (3.8<=)
- (特に)辞書のvalueに複数種類の型を許したいときに、厳密に型チェックを行える
    - そもそもそのようなユースケースでは`dataclasses.dataclass`を使うべきと思いますが...

In [43]:
#| echo: true
# これでは各キーがintとstr両方を取れてしまう
UserDic = dict[str, int | str]
user: UserDic = {"uid": 1, "name": "john"}
# これを型エラーにできない
user2: UserDic = {"uid": "1", "name": 123}

from typing import TypedDict
class User(TypedDict):
    uid: int
    name: str
user3: User = {"uid": 1, "name": "john"}
# 型エラー
user4: User = {"uid": "1", "name": 123}

## typing関連豆知識n連発{.smaller}
### `TypedDict` (3.8<=)
- (特に)辞書のvalueに複数種類の型を許したいときに、厳密に型チェックを行える
    - そもそもそのようなユースケースでは`dataclasses.dataclass`を使うべきと思いますが...

In [ ]:
#| echo: true
from typing import TypedDict
class User(TypedDict):
    uid: int
    name: str
user3: User = {"uid": 1, "name": "john"}
# 型エラー
user4: User = {"uid": "1", "name": 123}

- mypyでエラーとして検出できる
    - `error: Incompatible types (expression has type "str", TypedDict item "uid" has type "int")  [typeddict-item]`
    - `error: Incompatible types (expression has type "int", TypedDict item "name" has type "str")  [typeddict-item]`

## typing関連豆知識n連発{.smaller}
### `TypeGuard` (3.10<=)
- mypyやpyrightに無理やりtype narrowingをさせる機能
- 実装が間違っていても型チェッカは信じてしまうので諸刃の剣

In [48]:
#| echo: true
from typing import TypeGuard
def is_strs(vals: list[str] | list[int]) -> TypeGuard[list[str]]:
    return all(isinstance(v, str) for v in vals)
def is_ints(vals: list[str] | list[int]) -> TypeGuard[list[int]]:
    return all(isinstance(v, int) for v in vals)

def foo(vals: list[str] | list[int]):
    if all(isinstance(v, str) for v in vals):
        # ここで型エラーが出てしまう
        # vはstr | int型と認識される
        return [v.upper() for v in vals]

def bar(vals: list[str] | list[int]) -> list[str]:
    match vals:
        case vals if is_strs(vals):
            # vはstr型だと認識される
            return [v.upper() for v in vals]
        case vals if is_ints(vals):
            return [f"{v + 1}" for v in vals]
        case _:
            raise TypeError("invalid type")